In [ ]:
%%capture
%run "main.ipynb"
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
def cumulatively_categorise(column,threshold=0.75):
    threshold_value=int(threshold*len(column))
    categories_list=[]
    s=0
    counts=Counter(column)
    for i,j in counts.most_common():
        s+=dict(counts)[i]
        categories_list.append(i)
        if s>=threshold_value:
            break
    categories_list.append('Other')
    new_column=column.apply(lambda x: x if x in categories_list else 'Other')
    return new_column

## stm_prioriteit

In [ ]:
# NA's verwijderen omdat het er weinig zijn
df = df.dropna(subset=['stm_prioriteit'])

# veranderen van float naar categorie datatype
df['stm_prioriteit'] = df['stm_prioriteit'].astype('category')

df['stm_prioriteit'].value_counts()

De gemiddelde duur is wel degelijk verschillend bij de verschillende prioriteits klasses, zoals te zien in de output van de cel hieronder.

In [ ]:
mean_per_prio_lvl = df.groupby('stm_prioriteit')['anm_tot_fh'].mean()
mean_per_prio_lvl

In [ ]:
sns.boxplot(x='stm_prioriteit', y='anm_tot_fh', data=df, order=mean_per_prio_lvl.index)

plt.xlabel('Priority Level')
plt.ylabel('anm_tot_fh in minutes')

plt.show()

In [ ]:
df['stm_prioriteit'].info()
prioriteit_dummies = pd.get_dummies(df['stm_prioriteit'], prefix='prio')
prioriteit_dummies.info()

## stm_oorz_code

In [ ]:
df = df.dropna(subset=['stm_oorz_code'])
df['stm_oorz_code'] = df['stm_oorz_code'].astype('category')

In [ ]:
df['stm_oorz_code'].value_counts()

In [ ]:
oorz_code_dummies = pd.get_dummies(df['stm_oorz_code'], prefix='oorz_code')
oorz_code_dummies.info()

## stm_techn_gst

In [ ]:
df = df.dropna(subset=['stm_techn_gst'])
df['stm_techn_gst'].astype('category')
df['stm_techn_gst'].value_counts()

Het aantal categorieën wordt gereduceerd voordat er dummies gemaakt worden omdat het aantal kolommen ander te groot wordt

In [ ]:
df['stm_techn_gst'] = cumulatively_categorise(df['stm_techn_gst'], threshold=0.9)
df['stm_techn_gst'].value_counts()

In [ ]:
mean_per_tech_field = df.groupby('stm_techn_gst')['anm_tot_fh'].mean()
mean_per_tech_field

In [ ]:
sns.boxplot(x='stm_techn_gst', y='anm_tot_fh', data=df, order=mean_per_tech_field.index)

plt.xlabel('Techniek veld')
plt.ylabel('anm_tot_fh in minuten')

plt.show()

In [ ]:
techn_veld_dummies = pd.get_dummies(df['stm_techn_gst'], prefix='techn_veld')
techn_veld_dummies.info()

## stm_geo_mld

In [ ]:
df = df.dropna(subset=['stm_geo_mld'])
df['stm_geo_mld'].astype('category')
df['stm_geo_mld'].value_counts()

In [ ]:
geo_code_dummies = pd.get_dummies(df['stm_geo_mld'], prefix='geo_code')
geo_code_dummies.info()

## stm_contractgeb_mld

In [ ]:
df = df.dropna(subset=['stm_contractgeb_mld'])
df['stm_contractgeb_mld'].astype('category')
df['stm_contractgeb_mld'].value_counts()

In [ ]:
contractgebied_dummies = pd.get_dummies(df['stm_contractgeb_mld'], prefix='contractgebied')
contractgebied_dummies.info()

In [ ]:
df = df.reset_index(drop=True)
techn_veld_dummies = techn_veld_dummies.reset_index(drop=True)
oorz_code_dummies = oorz_code_dummies.reset_index(drop=True)
prioriteit_dummies = prioriteit_dummies.reset_index(drop=True)
geo_code_dummies = geo_code_dummies.reset_index(drop=True)
contractgebied_dummies = contractgebied_dummies.reset_index(drop=True)

features_to_use = [
    techn_veld_dummies
    # oorz_code_dummies, 
    # prioriteit_dummies, 
    # geo_code_dummies, 
    # contractgebied_dummies
    ]

model_df = pd.concat([df['anm_tot_fh'], *features_to_use], axis=1)
model_df = model_df.dropna()
model_df.sample(20)